# OkCupid: Dataset Analysis

This is a notebook I am using to test out the feasibility of using the [OkCupid Dataset](https://github.com/rudeboybert/JSE_OkCupid) for our first workshop.

In [1]:
#imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

#constants
%matplotlib inline
sns.set_style("dark")
sigLev = 3
figWidth = figHeight = 5

# Dataset Information

Before starting with the analysis of this dataset, it will be important to study the codebook of variables in ```data/JSE_OkCupid/okcupid_codebook.txt```. Some important notes I found from viewing this codebook:

* Contains Profiles from 25 Mile Radius outside of San Francisco, with at least one profile picture.

* Data Scraped in 2012.

* Contains $n = 59946$ observations; This is a mid-sized dataset, with a lot of information, although nont big-data unweildy.

* Contains a mix of different demographic and lifestyle data, along with some textual data in the ```essay``` variables.